<a href="https://colab.research.google.com/github/William-Schwarz/Decifra-Contrato/blob/william_ds/Decifra_Contrato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install google-genai google-adk

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
######################################################
# --- Agente 1: Receptor e Extrator de Contrato --- #
######################################################
def agente_receptor():
    receptor = Agent(
        name="agente_receptor",
        model=MODEL_ID,
        instruction="""
        Você é o agente inicial do sistema Decifra-Contrato. Sua tarefa é receber o contrato fornecido pelo usuário.
        Este contrato pode ser um texto diretamente inserido ou o conteúdo de um arquivo.

        Uma vez que o contrato é fornecido, sua principal função é extrair o texto completo e limpo do contrato.
        Remova qualquer formatação desnecessária que possa dificultar a análise posterior (como quebras de linha excessivas, numerações automáticas, etc.).

        Se o usuário fornecer informações adicionais sobre o tipo de contrato (por exemplo, "contrato de aluguel", "termo de serviço"), anote essa informação para auxiliar os próximos agentes.

        Ao final, entregue o texto puro do contrato para o próximo agente.
        """,
        description="Agente que recebe e extrai o texto do contrato."
    )
    return receptor

In [ ]:
#######################################################
# --- Agente 2: Identificador de Jargões --- #
#######################################################
def agente_identificador():
    identificador = Agent(
        name="agente_identificador",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em identificar jargões jurídicos e termos técnicos dentro de um texto de contrato.
        Sua tarefa é analisar o texto do contrato fornecido e listar todos os termos que provavelmente seriam desconhecidos por uma pessoa leiga.

        Considere termos como: latim jurídico (e.g., 'ad hoc'), termos técnicos específicos da área do contrato (e.g., em um contrato de tecnologia, 'API', 'SaaS'), e cláusulas com redação complexa.

        Apresente a lista de jargões identificados de forma clara e concisa para o próximo agente.
        """,
        description="Agente que identifica jargões jurídicos e técnicos."
    )
    return identificador

In [ ]:
########################################################
# --- Agente 3: Explicador de Termos --- #
########################################################
def agente_explicador(google_search):
    explicador = Agent(
        name="agente_explicador",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em explicar termos jurídicos e técnicos de forma clara e acessível para pessoas leigas.
        Você receberá uma lista de jargões identificados em um contrato.

        Para cada termo da lista, sua tarefa é fornecer uma explicação concisa e em linguagem simples.
        Se necessário, utilize a ferramenta de busca do Google (google_search) para encontrar definições e explicações adequadas.

        Tente fornecer exemplos práticos ou analogias para facilitar a compreensão.

        Apresente cada termo seguido de sua explicação de forma clara e organizada.
        """,
        description="Agente que explica jargões jurídicos e técnicos para leigos.",
        tools=[google_search]
    )
    return explicador

In [ ]:
#########################################################
# --- Agente 4: Resumidor e Destacador de Pontos Chave --- #
#########################################################
def agente_resumidor():
    resumidor = Agent(
        name="agente_resumidor",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em resumir contratos e destacar os pontos mais importantes para uma pessoa leiga.
        Com base no texto completo do contrato, sua tarefa é gerar um resumo conciso, com linguagem simples e direta.

        Além do resumo, identifique e liste as cláusulas ou pontos que são cruciais para o entendimento do usuário, como:
        - Objeto do contrato
        - Obrigações das partes
        - Prazos e condições
        - Formas de pagamento (se aplicável)
        - Condições de rescisão
        - Penalidades (se houver)

        Apresente o resumo e os pontos chave de forma clara e organizada, utilizando marcadores ou listas para facilitar a leitura.
        """,
        description="Agente que resume contratos e destaca pontos chave."
    )
    return resumidor

In [28]:
print("📜🧐 Iniciando o Sistema Decifra-Contrato 📜🧐")

# --- Obter o Contrato do Usuário ---
contrato_texto = input("❓ Por favor, cole o texto do contrato aqui: ")

if not contrato_texto:
    print("Você precisa fornecer o texto do contrato!")
else:
    print("\n✨ Processando o contrato. Aguarde...\n")

    # Inicializa os agentes
    receptor = agente_receptor()
    identificador = agente_identificador()
    explicador = agente_explicador(google_search)
    resumidor = agente_resumidor()

    # --- Execução dos Agentes ---

    # Agente Receptor
    texto_contrato_extraido = call_agent(receptor, contrato_texto)
    print("--- 📝 Contrato Extraído ---")
    display(to_markdown(texto_contrato_extraido))
    print("-----------------------------\n")

    # Agente Identificador de Jargões
    jargoes_identificados = call_agent(identificador, texto_contrato_extraido)
    print("--- 🔎 Jargões Identificados ---")
    display(to_markdown(jargoes_identificados))
    print("-------------------------------\n")

    # Agente Explicador de Termos
    explicacoes_termos = call_agent(explicador, jargoes_identificados)
    print("--- 💡 Explicações dos Termos ---")
    display(to_markdown(explicacoes_termos))
    print("---------------------------------\n")

    # Agente Resumidor e Destacador de Pontos Chave
    resumo_contrato = call_agent(resumidor, texto_contrato_extraido)
    print("--- 🔑 Resumo e Pontos Chave do Contrato ---")
    display(to_markdown(resumo_contrato))
    print("-------------------------------------------\n")

    print("✅ Análise do Contrato Concluída!")

📜🧐 Iniciando o Sistema Decifra-Contrato 📜🧐
❓ Por favor, cole o texto do contrato aqui: CONTRATO DE TRABALHO POR EXPERIÊNCIA  Por este instrumento particular, que entre si fazem CLOUD APOIO ADMINISTRATIVO LTDA, inscrita no CNPJ sob o n  ∘   48.772.069/0001-60, com sede à Avenida CORONEL TRAVASSOS, 1992 SALA 203, bairro BOA VISTA, municipio de Novo Hamburgo/RS, doravante denominada EMPREGADORA, e de outro lado JAMILEH FLACH BRAGA, brasileira, solteira, inscrito(a) no CPF sob o n  ∘   019.127.350-32, RG n  ∘   e CTPS 0191273-5032/RS, residente e domiciliado(a) à Rua ASSIS CHATEAUBRIAND, 3, bairro LIBERDADE, municipio de Novo Hamburgo/RS.  doravante denominado EMPREGADO, fica celebrado o presente CONTRATO DE TRABALHO EM CARÁTER DE EXPERIÊNCIA, de acordo com a legislação em vigor e as condições especificadas a seguir.  Cláusula 1  a  : Fica o EMPREGADO admitido no quadro de trabalhadores da EMPREGADORA para exercer o cargo de ANALISTA DE RH (CBO 252405), com remuneração de R$ 2.200,00 (dois

> Contrato de Trabalho por Experiência
> 
> Por este instrumento particular, que entre si fazem CLOUD APOIO ADMINISTRATIVO LTDA, inscrita no CNPJ sob o n ∘ 48.772.069/0001-60, com sede à Avenida CORONEL TRAVASSOS, 1992 SALA 203, bairro BOA VISTA, município de Novo Hamburgo/RS, doravante denominada EMPREGADORA, e de outro lado JAMILEH FLACH BRAGA, brasileira, solteira, inscrito(a) no CPF sob o n ∘ 019.127.350-32, RG n ∘ e CTPS 0191273-5032/RS, residente e domiciliado(a) à Rua ASSIS CHATEAUBRIAND, 3, bairro LIBERDADE, município de Novo Hamburgo/RS, doravante denominado EMPREGADO, fica celebrado o presente CONTRATO DE TRABALHO EM CARÁTER DE EXPERIÊNCIA, de acordo com a legislação em vigor e as condições especificadas a seguir.
> 
> Cláusula 1 a : Fica o EMPREGADO admitido no quadro de trabalhadores da EMPREGADORA para exercer o cargo de ANALISTA DE RH (CBO 252405), com remuneração de R$ 2.200,00 (dois mil, duzentos reais) por mês, a ser pago mensalmente até o prazo previsto na legislação
> Parágrafo Único: Reserva-se a EMPREGADORA o direito de proceder a transferência do empregado para outro cargo ou função que entenda que este demonstre melhor capacidade de adaptação desde que compatível com sua condição pessoal.
> 
> Cláusula 2 a . O local de trabalho do EMPREGADO será em quaisquer dos estabelecimentos da EMPREGADORA no território nacional, podendo ocorrer a transferência para outra localidade na forma da legislação.
> 
> Cláusula 3 a : O EMPREGADO cumprirá o seguinte horário de trabalho
> 
> Dia da Semana Horário Entrada Início Intervalo Fim Intervalo Horário Saida
> Domingo Folga
> Segunda 08.00.00 12:00:00 13.00 00 18.00.00
> Terça 08:00 00 12.00.00 13.00.00 18:00.00
> Quarta 08:00:00 12.00.00 13.00.00 18.00.00
> Quinta 08:00.00 12:00:00 13:00.00 18. 00 00
> Sexta 08.00:00 12.00.00 13.00.00 17.00.00
> Sábado
> 
> Horas p/semana: 44,00
> Dia variável Não
> Tempo parcial Não é tempo parcial
> 
> Parágrafo Único: Se houver horas extras, estas serão pagas na forma da lei ou, ainda, compensadas com repouso correspondente, conforme determinação de convenção ou acordo coletivo de trabalho ou, ainda, acordo individual, se firmado.
> 
> Cláusula 4 a O presente contrato tem caráter de experiência vigorando a partir da data de assinatura do presente contrato até 04/07/2024, em cujo termo estará extinto, sem que caiba a qualquer das partes aviso prévio ou indenização
> 
> Parágrafo 1 ∘ Este contrato poderá ser prorrogado uma única vez, observando o limite máximo referido no art. 445, parágrafo único da CLT.
> 
> Parágrafo 2 ∘ O presente contrato está respaldado pela cláusula assecuratória do direito reciproco de rescisão, prevista no art. 480 da CLT.
> 
> Parágrafo 3 ∘ : Conforme o art. 487, parágrafo 2 ∘ da CLT, o empregado que pedir demissão e não trouxer documento que comprove que achou outro emprego e/ou se recusar a trabalhar os 30 dias, poderá ter descontado o período do aviso prévio.
> 
> Cláusula 5 a Permanecendo o EMPREGADO a serviço da EMPREGADORA após o término da experiência, continuarão em vigor as cláusulas deste instrumento, embora o contrato passe a vigorar por tempo indeterminado.
> 
> Cláusula 6 ∘ : Caso o EMPREGADO faça uso de transporte regular público em seus deslocamentos residência/trabalho e vice-versa, deverá solicitar a EMPREGADORA, por escrito e contra-recibo, o fornecimento de vale-transporte.
> 
> Cláusula 7 ∘ , O EMPREGADO autoriza o desconto em seu salário das importâncias que lhe forem adiantadas pelo empregador, bem como aos descontos legais, de alimentação, habitação e vale transporte.
> 
> Parágrafo Único: O EMPREGADO, sempre que causar algum prejuizo ao empregador, resultante de qualquer conduta dolosa ou culposa, ficará obrigado a ressarcir a EMPREGADORA por todos os danos causados, pelo que, desde já, fica a EMPREGADORA, autorizada a efetivar o desconto da importância correspondente ao prejuízo, o qual fará, com fundamento no parágrafo 1 c artigo 462 da CLT. do
> 
> Cláusula 8 ∘ : O EMPREGADO compromete-se também a respeitar o Regulamento Interno da empresa e que a violação deste implicará em sanção, cuja graduação dependerá da gravidade da mesma, podendo culminar na rescisão do contrato de trabalho.
> 
> Cláusula 9 a : Para fins de atendimento a Lei Geral de Proteção de Dados, o EMPREGADO tem ciência e manifesta o seu consentimento quanto ao uso de quaisquer de seus dados pessoais pelo EMPREGADOR, para que este faça uso destes nos procedimentos necessários durante a relação de emprego entre as partes, tais como a admissão, a manutenção da relação de emprego, o desligamento, a medicina ocupacional, a relação com planos de saúde, a manutenção de convênios com terceiros, ao cadastro e uso de biometria, as operações que envolvam informações bancária, a consulta em redes sociais, entre outros que envolvam a relação de trabalho.
> 
> E, por estarem de pleno acordo, as partes assinam este instrumento em duas vias de igual teor na presença das testemunhas abaixo relacionadas.
> 
> CLOUD APOIO ADMINISTRATIVO LTDA
> 
> Novo Hamburgo, RS, 21 de maio de 2024.
> 
> mildchraga
> JAMILEH FLACH BRAGA
> 
> Testemunhas:
> 
> Nome:
> CPF/RG:
> Nome:
> CPF/RG
> 
> Este é um contrato de trabalho por experiência.
> 


-----------------------------

--- 🔎 Jargões Identificados ---


> Com base no texto do contrato de trabalho por experiência fornecido, listo os seguintes jargões jurídicos e termos técnicos que podem ser desconhecidos por uma pessoa leiga:
> 
> 1.  **"Inscrita no CNPJ sob o n ∘..."**: Refere-se ao Cadastro Nacional de Pessoas Jurídicas, um registro obrigatório para empresas no Brasil.
> 2.  **"Residente e domiciliado(a)"**: Termo jurídico que indica o local onde a pessoa vive e é legalmente reconhecida como seu endereço principal.
> 3.  **"Contrato de Trabalho em Caráter de Experiência"**: Tipo específico de contrato de trabalho com duração determinada.
> 4.  **"CBO 252405"**: Classificação Brasileira de Ocupações, um código que padroniza e identifica a profissão do empregado.
> 5.  **"Legislação em vigor"**: Refere-se às leis e normas atuais que regem as relações de trabalho.
> 6.  **"Território nacional"**: Refere-se a todo o espaço geográfico do Brasil.
> 7.  **"Na forma da lei"**: Significa que a ação deve seguir o que está determinado nas leis existentes.
> 8.  **"Horas extras"**: Tempo de trabalho que excede a jornada normal estabelecida.
> 9.  **"Convenção ou acordo coletivo de trabalho"**: Negociações entre sindicatos e empregadores que estabelecem condições de trabalho.
> 10. **"Aviso prévio"**: Notificação antecipada de rescisão do contrato de trabalho.
> 11. **"Indenização"**: Compensação financeira paga em caso de rescisão do contrato.
> 12. **"Art. 445, parágrafo único da CLT"**: Artigo da Consolidação das Leis do Trabalho que trata da duração do contrato de experiência.
> 13. **"Cláusula assecuratória do direito reciproco de rescisão"**: Permite a rescisão antecipada do contrato de experiência, garantindo direitos em caso de rompimento.
> 14. **"Art. 480 da CLT"**: Artigo da Consolidação das Leis do Trabalho que trata da rescisão antecipada do contrato de experiência por parte do empregado.
> 15. **"Art. 487, parágrafo 2 ∘ da CLT"**: Artigo da Consolidação das Leis do Trabalho que trata das consequências do não cumprimento do aviso prévio pelo empregado.
> 16. **"Vale-transporte"**: Benefício pago para custear o deslocamento do empregado entre residência e trabalho.
> 17. **"Conduta dolosa ou culposa"**: Ações intencionais (dolosa) ou por negligência (culposa) que causam prejuízo.
> 18. **"Ressarcir"**: Compensar ou indenizar por um dano ou prejuízo causado.
> 19. **"Artigo 462 da CLT"**: Artigo da Consolidação das Leis do Trabalho que trata dos descontos permitidos no salário.
> 20. **"Regulamento Interno"**: Conjunto de normas e regras estabelecidas pela empresa.
> 21. **"Rescisão do contrato de trabalho"**: Encerramento do contrato de trabalho.
> 22. **"Lei Geral de Proteção de Dados"**: Lei que regulamenta o tratamento de dados pessoais.
> 23. **"Relação de emprego entre as partes"**: Refere-se ao vínculo empregatício entre empregador e empregado.
> 24. **"Medicina ocupacional"**: Área da medicina que cuida da saúde dos trabalhadores.
> 
> 


-------------------------------

--- 💡 Explicações dos Termos ---


> Ok, aqui estão as explicações dos termos jurídicos e técnicos que você listou, de forma clara e acessível:
> 
> 1.  **"Inscrita no CNPJ sob o n ∘..."**: É como se fosse o "RG" da empresa. O CNPJ é um número que identifica a empresa perante a Receita Federal e garante que ela está regularizada para exercer suas atividades.
> 2.  **"Residente e domiciliado(a)"**: Significa o endereço oficial da pessoa, onde ela mora e é encontrada para questões legais. É como se fosse o endereço que vale para tudo, desde receber contas até ser notificado pela justiça.
> 3.  **"Contrato de Trabalho em Caráter de Experiência"**: É um contrato por tempo limitado, geralmente de 90 dias, que serve para o empregador e o empregado se conhecerem melhor. É como um "test drive" antes de uma possível contratação definitiva.
> 4.  **"CBO 252405"**: É um código que diz qual é a sua profissão. Imagine que cada profissão tem um número para ser identificada facilmente em cadastros e documentos oficiais.
> 5.  **"Legislação em vigor"**: São as leis que estão valendo atualmente e que devem ser seguidas. É como se fossem as "regras do jogo" que todos devem cumprir.
> 6.  **"Território nacional"**: É todo o espaço geográfico do Brasil, incluindo terras, águas e espaço aéreo. É como se fosse o "país inteiro".
> 7.  **"Na forma da lei"**: Quer dizer que algo deve ser feito seguindo exatamente o que as leis dizem. É como seguir as instruções de um manual para não fazer errado.
> 8.  **"Horas extras"**: São as horas trabalhadas além da jornada normal. Se você trabalha mais do que o combinado, essas horas a mais são chamadas de horas extras e devem ser pagas com um valor adicional.
> 9.  **"Convenção ou acordo coletivo de trabalho"**: São acordos feitos entre os sindicatos dos trabalhadores e as empresas para definir regras sobre salários, benefícios e condições de trabalho. É como se fosse um "contrato" feito para toda uma categoria de trabalhadores.
> 10. **"Aviso prévio"**: É uma comunicação antecipada da intenção de rescindir o contrato de trabalho. Se a empresa vai te demitir ou você vai pedir demissão, é preciso avisar com antecedência, que geralmente é de 30 dias.
> 11. **"Indenização"**: É um valor pago para compensar um dano ou prejuízo. Por exemplo, se você é demitido sem justa causa, pode ter direito a uma indenização.
> 12. **"Art. 445, parágrafo único da CLT"**: É um artigo da lei que diz que o contrato de experiência não pode durar mais que 2 anos. É como se fosse o limite máximo de tempo para esse tipo de contrato.
> 13. **"Cláusula assecuratória do direito reciproco de rescisão"**: É uma permissão para que o contrato de experiência seja terminado antes do prazo, tanto pelo empregador quanto pelo empregado, garantindo alguns direitos.
> 14. **"Art. 480 da CLT"**: É um artigo da lei que fala sobre o que acontece se o empregado pedir demissão antes do fim do contrato de experiência. Nesse caso, ele pode ter que pagar uma indenização para a empresa.
> 15. **"Art. 487, parágrafo 2 ∘ da CLT"**: É um artigo da lei que fala sobre o que acontece se o empregado não cumprir o aviso prévio. Nesse caso, ele pode ter o valor correspondente aos dias não trabalhados descontado do seu salário.
> 16. **"Vale-transporte"**: É um benefício pago pelo empregador para ajudar o empregado a pagar as despesas com transporte entre a casa e o trabalho.
> 17. **"Conduta dolosa ou culposa"**: É quando alguém faz algo de propósito (dolosa) ou por falta de cuidado (culposa) que causa um prejuízo. É como se fosse uma ação errada que pode ter consequências.
> 18. **"Ressarcir"**: Significa pagar de volta ou compensar um prejuízo que você causou. É como se fosse "reparar o estrago".
> 19. **"Artigo 462 da CLT"**: É um artigo da lei que diz quais são os descontos que podem ser feitos no salário do empregado, como, por exemplo, o INSS e o imposto de renda.
> 20. **"Regulamento Interno"**: É um conjunto de regras da empresa que todos os empregados devem seguir. É como se fosse um "manual de conduta" da empresa.
> 21. **"Rescisão do contrato de trabalho"**: É o término do contrato de trabalho, seja por demissão, pedido de demissão ou outro motivo.
> 22. **"Lei Geral de Proteção de Dados"**: É uma lei que protege os seus dados pessoais e diz como as empresas podem usá-los. É como se fosse uma "guarda-costas" dos seus dados.
> 23. **"Relação de emprego entre as partes"**: É o vínculo entre o empregador e o empregado, que é regido pelas leis trabalhistas.
> 24. **"Medicina ocupacional"**: É a área da medicina que cuida da saúde dos trabalhadores, prevenindo doenças e acidentes no trabalho.
> 
> Espero que estas explicações tenham sido úteis!
> 


---------------------------------

--- 🔑 Resumo e Pontos Chave do Contrato ---


> ## Resumo do Contrato de Trabalho por Experiência
> 
> Este contrato define os termos do emprego de Jamileh Flach Braga como Analista de RH na CLOUD APOIO ADMINISTRATIVO LTDA. O contrato é por experiência, com duração determinada, e estabelece o salário, horário de trabalho, local de trabalho e outras condições importantes.
> 
> ## Pontos Chave do Contrato
> 
> *   **Objeto do Contrato:** Contratação de Jamileh Flach Braga como Analista de RH.
> *   **Empregador:** CLOUD APOIO ADMINISTRATIVO LTDA.
> *   **Empregado:** Jamileh Flach Braga.
> *   **Cargo:** Analista de RH (CBO 252405).
> *   **Salário:** R$ 2.200,00 por mês.
> *   **Duração do Contrato:** A partir de 21 de maio de 2024 até 04 de julho de 2024.
>     *   Possibilidade de prorrogação por mais um período.
> *   **Horário de Trabalho:**
>     *   Segunda a Quinta: 08:00 às 12:00 e 13:00 às 18:00.
>     *   Sexta: 08:00 às 12:00 e 13:00 às 17:00.
>     *   Total de 44 horas semanais.
> *   **Local de Trabalho:** Qualquer estabelecimento da empregadora no território nacional.
> *   **Obrigações do Empregado:**
>     *   Cumprir o horário de trabalho.
>     *   Respeitar o regulamento interno da empresa.
>     *   Informar necessidade de vale-transporte.
>     *   Ressarcir a empresa por prejuízos causados por conduta dolosa ou culposa.
> *   **Rescisão:**
>     *   O contrato se extinguirá automaticamente ao final do período de experiência (04/07/2024).
>     *   Caso o empregado peça demissão antes do prazo, poderá ter descontado o valor correspondente ao aviso prévio, caso não comprove ter outro emprego ou se recuse a trabalhar durante o período do aviso.
> *   **Autorizações:** O empregado autoriza descontos no salário referentes a adiantamentos, descontos legais, alimentação, habitação, vale-transporte e prejuízos causados à empresa. Também autoriza o uso de seus dados pessoais pela empresa para fins relacionados ao contrato de trabalho, conforme a Lei Geral de Proteção de Dados (LGPD).


-------------------------------------------

✅ Análise do Contrato Concluída!
